In [ ]:
import pandas as pd
from app.DataTransformer import DataTransformer

In [ ]:
data_transformer = DataTransformer(date='2024-02-25')

In [ ]:
data_transformer.transformed_data_paths['Sony_Xperia_XA']

In [ ]:
pd.read_csv('/Users/kuba/SkiApp/data/2023-12-31/Sony_Xperia_XA/final_df.csv')

In [ ]:
data_transformer.transformed_data

In [ ]:
set(data_transformer.device_dataframes['Sony_Xperia_M5'].keys())

In [ ]:
from app.device_namespace import FINAL_DF_SENSORS
FINAL_DF_SENSORS

In [ ]:
set(FINAL_DF_SENSORS) - set(data_transformer.device_dataframes['Sony_Xperia_M5'].keys())

In [ ]:
FINAL_DF_SENSORS

In [ ]:
from pathlib import Path
from app.device_namespace import DATA_PATH
def generate_path_for_final_df(date: str, device: str) -> Path:
    """Generate path for final dataframe"""
    return Path(DATA_PATH, date, device, "final_df.csv")

In [ ]:
for device in data_transformer.transformed_data_paths.keys():
    print(device)
    print(generate_path_for_final_df(data_transformer.date, device))
    print()

In [ ]:
data_transformer.device_dataframes['HONOR_8X']['TotalAcceleration']['time'][0]

In [ ]:
temp = data_transformer.device_dataframes['HONOR_8X']['Magnetometer']
temp

In [ ]:
data_transformer.device_dataframes['HONOR_8X']

In [ ]:
temp = data_transformer.device_dataframes['HONOR_8X']

In [ ]:
temp['time'] = pd.to_datetime(temp['time'])

In [ ]:
type(temp['time'][0])

In [ ]:
import copy
def cut_dataframes_inside_device(device_dataframes: dict,) -> dict:
    """Cut dataframes to same length"""
    device_dataframes_copy = copy.deepcopy(device_dataframes)
    min_time = max_time = None

    # Find the global min and max time
    for df in device_dataframes_copy.values():
        current_min = df['time'].min()
        current_max = df['time'].max()
        min_time = current_min if min_time is None or current_min > min_time else min_time
        max_time = current_max if max_time is None or current_max < max_time else max_time
    print(f"min_time: {min_time}, max_time: {max_time}")

    # Cut each dataframe to the global min and max time
    for key, df in device_dataframes_copy.items():
        device_dataframes_copy[key] = df[(df['time'] >= min_time) & (df['time'] <= max_time)]

    return device_dataframes_copy

In [ ]:
#TODO: need adjustment for example if the value is missing in the beginign it will put nan
def fill_missing_times(device_dataframes: dict) -> dict:
    """Fill missing times in dataframes"""
    device_dataframes_copy = copy.deepcopy(device_dataframes)
    all_times = set()

    # Get all unique times
    for df in device_dataframes_copy.values():
        all_times.update(df['time'].unique())

    # Fill missing times and interpolate other columns
    for key, df in device_dataframes_copy.items():
        unique_times = set(df['time'].unique())
        missing_times = all_times - unique_times
        if missing_times:
            missing_df = pd.DataFrame(missing_times, columns=['time'])
            df = pd.concat([df, missing_df])
            df.sort_values('time', inplace=True)
            df.set_index('time', inplace=True)
            df = df.interpolate(method='time')
            device_dataframes_copy[key] = df.reset_index()

    return device_dataframes_copy

In [ ]:
new_data = cut_dataframes_inside_device(data_transformer.device_dataframes['HONOR_8X'])

In [ ]:
filled_missing_time_data = fill_missing_times(new_data)

In [ ]:
print("new data")
for key in new_data.keys():
    print(key)
    print(new_data[key].size)
    print()
print("filled_missing_time_data")
for key in filled_missing_time_data.keys():
    print(key)
    print(filled_missing_time_data[key].size)
    print()

In [ ]:
filled_missing_time_data['TotalAcceleration']

In [ ]:
def prepare_final_df(device_dataframes: dict) -> pd.DataFrame:
    """Prepare final dataframe that merges all dataframes"""
    dfs = []

    # For each dataframe, create a single column where the values are lists of the values in the original columns
    for key, df in device_dataframes.items():
        df = df.drop(columns='seconds_elapsed', errors='ignore')
        df.set_index('time', inplace=True)
        df[key] = df.values.tolist()
        df = df[[key]]  # Keep only the new column
        dfs.append(df)

    # Merge all dataframes on the index
    final_df = pd.concat(dfs, axis=1)

    return final_df

In [ ]:
final = prepare_final_df(filled_missing_time_data)

In [ ]:
final

In [ ]:
final.columns

In [ ]:
for key in new_data.keys():
    print(key)
    print(new_data[key].size)
    print()


In [ ]:

for key in filled_missing_time_data.keys():
    print(key)
    print(filled_missing_time_data[key].size)
    print()

In [ ]:
new_data['Magnetometer']

In [ ]:
list(data_transformer.transformed_data['Sony_Xperia_M5'].values())[0]

In [ ]:
data_transformer.transformed_data['Sony_Xperia_M5']

In [ ]:
pd.read_csv('data/2023-12-31/Sony_Xperia_M5/Rules.csv')

In [ ]:
pd.read_csv(list(data_transformer.transformed_data['Sony_Xperia_M5'].values())[0]).head(20)

In [ ]:
def match_times_acros_sensors(device_sensor: dict):


In [ ]:
#load file to pd dataframe /Users/kuba/SkiApp/input_data/2023-12-31/HONOR_8X/Accelerometer.csv
df = pd.read_csv('/Users/kuba/SkiApp/input_data/2023-12-31/HONOR_8X/Accelerometer.csv', sep=',')

In [ ]:
df

In [ ]:
pd.to_datetime(df['time'], unit='ns')

In [ ]:
df_time = df.copy()
df_time['time'] = pd.to_datetime(df['time'], unit='ns')  # use 'ms' if your timestamps are in milliseconds

In [ ]:
type(df_time['time'][1])

In [ ]:
def apply_offset(df: pd.DataFrame, offset: int):
    #add offset in ns to time column where offset is ms and time is pandas._libs.tslibs.timestamps.Timestamp
    df['time'] = df['time'] + pd.Timedelta(offset, unit='ms')
    return df

In [ ]:
df_time

In [ ]:
df_offset = apply_offset(df_time, 1)
df_offset

In [ ]:
df2= pd.read_csv('/Users/kuba/SkiApp/input_data/2023-12-30/Sony_Xperia_XA/TotalAcceleration.csv', sep=',')

In [ ]:
pd.to_datetime(df2['time'], unit='ns')  # use 'ms' if your timestamps are in milliseconds

In [ ]:
# load text file /Users/kuba/SkiApp/input_data/2023-12-31/OPPO_AX7/offset.txt and read first file to int
f = open('/Users/kuba/SkiApp/input_data/2023-12-31/OPPO_AX7/offset.txt', 'r')
offset = int(f.readline())
offset

In [ ]:
import os
from pathlib import Path

# Original path
path = "/Users/kuba/SkiApp/input_data/2023-12-30/Sony_Xperia_XA/TotalAcceleration.csv"

# Modify the path
new_path = Path(path).relative_to("/Users/kuba/SkiApp/input_data")

In [ ]:
new_path = os.path.join("data", new_path)
new_path

In [ ]:
path = Path("/Users/kuba/SkiApp/input_data/2023-5-30/Soy_Xperia_bA/ToalAccpeleratin.csv")
last_parts = path.parts[-3:]

final_path =  Path('data', *last_parts)
folders = final_path.parts[:-1]
folders = Path(*folders)
print(final_path)
print(folders)
folders.mkdir(parents=True, exist_ok=True)
df.to_csv(final_path, index=False)

In [ ]:
final_path.mkdir(parents=True, exist_ok=True)

In [ ]:
df.to_csv(final_path, index=False)

In [ ]:
path

In [ ]:
new_date_directory = Path('/data/2023-12-30/Sony_Xperia_XA/TotalAcceleration.csv')


In [ ]:
parent_directory = new_date_directory.parent

# Get the last three components
last_three_components = parent_directory.parts[-3:]

# Join the components back together into a Path
last_three_path = Path(*last_three_components)

print(last_three_path)

In [ ]:
new_date_directory = Path('/data/2023-12-30/Sony_Xperia_XA/TotalAcceleration.csv')

new_date_directory.mkdir(parents=True, exist_ok=True)

In [ ]:
os.makedirs(os.path.dirname('data/2023-12-30/Sony_Xperia_XA'), exist_ok=True)

df.to_csv(new_path, index=False)